In [2]:
import os
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pprint import pprint
%matplotlib inline

# profile

# mean

In [ ]:
bw_scan = False

import subprocess

name = 'atac_OCR_specific'
capture_regions_file = 'atac_validation/atac_wt_OCR_specific_peaks_Pou5f3.narrowPeak'
bigwig_files = [
    'atat_signal/atac_wt_oblong.bw',
    'atat_signal/atac_ko_pou5f3_oblong.bw',
    #'atac_MO_control_GR2018.bw', 'atac_MO_nanog_GR2018.bw',
    #'atac_MO_nanog_rescue_GR2018.bw'
]
labels = [
    'wt',
    'ko',
    #'mo_control', 'mo', 'mo_resuce',
]

cmd = f'cut -f 1-4 {capture_regions_file} > atac_validation/captures_regions.bed'
subprocess.call(cmd, shell=True)

for label, bigwig_file in zip(labels, bigwig_files):
    bw_scan_cmd = f'bigWigAverageOverBed {bigwig_file} atac_validation/captures_regions.bed atac_validation/{name}_{label}_signal.tsv'
    if bw_scan:
        print(subprocess.check_output(bw_scan_cmd.split()).decode(), end='')

def get_bigwig_mean(bigwig_file):
    import subprocess
    fold = subprocess.check_output(f'bigWigInfo {bigwig_file} | grep mean',
                                   shell=True).decode()
    fold = float(fold.split()[1])
    return fold

capture_regions = pd.read_csv('atac_validation/captures_regions.bed', sep='\t', header=None)
capture_signal_atac_OCR_specific = pd.DataFrame(index=capture_regions[3].to_list())
for label, bigwig_file in zip(labels, bigwig_files):
    avg = get_bigwig_mean(bigwig_file)
    capture_signal_atac_OCR_specific[label] = pd.read_csv(
        f'atac_validation/{name}_{label}_signal.tsv',
        sep='\t',
        header=None,
        index_col=0,
        names=['size', 'covered', 'sum', 'mean0', 'mean'])['mean0'] / avg

capture_signal_atac_OCR_specific